# Beer or Wine: the impact of climate change on Bamberg's vineyards
# Research Question
### Can Franconia's wine region now include Bamberg due to warmer weather?
Despite the negative effects of climate change, some wine varietals thrive in warmer climates. For instance, Bamberg, once a wine-producing region, could potentially regain its status due to rising temperatures. We compare the climate of nearby Würzburg, at the heart of the Franconian winemaking region, to Bamberg's which is currently just outside of the Franconian winemaking region.

## Map of Germany's Winemaknig Regions
![Map of Germany's Winemaknig Regions](https://upload.wikimedia.org/wikipedia/commons/8/89/WeinbaugebieteDeutschland.svg)

## Data Sources
We chose to use station data from the Deutscher Wetterdienst (DWD) because it has many direct measures of weather (ex. air temperature 2 meters above ground) and is accurate, easily accessible, and the DWD works to ensure historical data is integrated and well-maintained.

The data we'll be using are the following (more information can be found [here](https://wetterdienst.readthedocs.io/en/latest/data/coverage/dwd/observation/monthly.html)):
- QN_4 - quality level of the data in the following columns
- MO_N - monthly mean of cloud cover
- MO_TT - monthly mean of daily temperature means in 2m height
- MO_TX - monthly mean of daily temperature maxima in 2m height
- MO_TN - monthly mean of daily temperature minima in 2m height
- MO_FK - monthly mean of daily wind speed
- MX_TX - monthly maximum of daily temperature maxima in 2m height
- MX_FX - monthly maximum of daily wind speed
- MX_TN - monthly minimum of daily temperature minima in 2m height
- MO_SD_S - monthly sum of sunshine duration
- QN_6 - quality level of the data in the following columns
- MO_RR - monthly sum of precipitation height
- MX_RS - monthly maximum of daily precipitation height

### Datasource1: Deutscher Wetterdienst (DWD) Station 282 Bamberg
* Metadata URL: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/monthly/kl/historical/DESCRIPTION_obsgermany_climate_monthly_kl_historical_en.pdf
* Data URL: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/monthly/kl/historical/monatswerte_KL_00282_18810101_20231231_hist.zip
* Data Type: TXT

This data is from the German weather service DWD at node 282 in Bamberg. We'll be using the air temperature monthly aggregate from 1949 - 2024.

### Datasource2: Deutscher Wetterdienst (DWD) Station 5705 Würzburg
* Metadata URL: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/monthly/kl/historical/DESCRIPTION_obsgermany_climate_monthly_kl_historical_en.pdf
* Data URL: https://opendata.dwd.de/climate_environment/CDC/observations_germany/climate/monthly/kl/historical/monatswerte_KL_05705_18810101_20231231_hist.zip
* Data Type: TXT

This data is from the German weather service DWD at node 5705 in Würzburg. We'll be using the air temperature monthly aggregate from 1949 - 2024.

### Data Source License
The license information can be found in German [here](https://opendata.dwd.de/LIESMICH.txt) and in English [here](https://www.dwd.de/EN/service/legal_notice/legal_notice.html). The weather data is offered under a [Creative Commons 4.0 license](https://creativecommons.org/licenses/by/4.0/). We will fulfill the obligations to name the data source used in a manner which complies with [these guidelines](https://www.dwd.de/EN/service/legal_notice/templates_dwd_as_source.html?nn=450678).

## Data Pipeline
### Overview
We used Python with the urllib.request library for downloading the dataset from the DWD website, the zipfile library to extract the .txt file from the dowloaded ZIP file, and pandas for temporarily storing and transforming the data.

### Transformations
The following transformation steps were applied:
1. Loading the CSV (in .txt form) into pandas converted data types into appropriate values like int64 and float64
2. The "eor" (end of record) column was removed as that's not necessary for our purposes.
3. Missing or invalid data are marked with -999 in the dataset. We replace these with pd.NA (NA values) since we don't want to be able to compare, numerically, valid and invalid data at all.

### Problems
We first attempted to use Jayvee to create the pipeline but ran into issues with formatting. Specifically, it was too difficult to remove (especially varying amounts of) preceding whitespace from a text file which separated values by with a character. This also prevented us from changing the value types from string to integer or decimal. We were also unable to chain multiple transformations, making this issue too time consuming to resolve with Jayvee.

The issue was resolved when we switched to Python using additional libraries.

### Errors and Updates
The DWD has a fixed format for files and data, so we don't expect any deviation from the current format unless DWD standards change.

We do check to see if the file was downloaded, and the data extracted correctly. If not, the process is halted. It's worthy of note that the DWD also has a standardized file naming structure.

The data we are using is updated regularly (at least monthly). No assumptions were made regarding the size of the data we are processing so additional or updated rows should not affect the pipeline negatively. The addition of an entirely new category of data (a new column) would go unused but should not break the pipeline because of how pandas dataframes work.

## Results and Limitations of the Pipeline

In [8]:
import pandas as pd

from FranconianWeather import download_file
from FranconianWeather import load_txt_from_zip
from FranconianWeather import clean_df
from FranconianWeather import save_to_sqlite

In [9]:
bamberg = "monatswerte_KL_00282_18810101_20231231_hist.zip"
wuerzburg = "monatswerte_KL_05705_18810101_20231231_hist.zip"

download_file(bamberg)
download_file(wuerzburg)

File downloaded successfully!
File downloaded successfully!


In [10]:
# Define the name of the text file inside the zip file
bamberg_data_file = "produkt_klima_monat_18810101_20231231_00282.txt"

# Load the text file into a pandas DataFrame
bamberg_df = load_txt_from_zip(bamberg, bamberg_data_file)

bamberg_df

,STATIONS_ID,MESS_DATUM_BEGINN,MESS_DATUM_ENDE,QN_4,MO_N,MO_TT,MO_TX,MO_TN,MO_FK,MX_TX,MX_FX,MX_TN,MO_SD_S,QN_6,MO_RR,MX_RS,eor
0,282,18810101,18810131,5,-999.00,-999.00,-2.70,-999.00,-999.00,-999.0,-999.0,-999.0,-999.00,-999,-999.0,-999.0,eor
1,282,18810201,18810228,5,-999.00,-999.00,3.50,-999.00,-999.00,-999.0,-999.0,-999.0,-999.00,-999,-999.0,-999.0,eor
2,282,18810301,18810331,5,-999.00,-999.00,7.20,-999.00,-999.00,-999.0,-999.0,-999.0,-999.00,-999,-999.0,-999.0,eor
3,282,18810401,18810430,5,-999.00,-999.00,10.40,-999.00,-999.00,-999.0,-999.0,-999.0,-999.00,-999,-999.0,-999.0,eor
4,282,18810501,18810531,5,-999.00,-999.00,17.80,-999.00,-999.00,-999.0,-999.0,-999.0,-999.00,-999,-999.0,-999.0,eor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711,282,20230801,20230831,3,5.39,18.91,24.93,13.25,1.35,33.2,14.9,6.6,180.33,3,84.2,16.6,eor
1712,282,20230901,20230930,3,3.71,16.44,24.96,9.10,1.17,31.0,12.9,3.4,246.97,3,19.1,7.5,eor
1713,282,20231001,20231031,3,6.45,11.64,17.54,6.34,1.52,26.7,16.0,-2.4,108.75,3,62.8,8.3,eor
1714,282,20231101,20231130,3,7.03,5.97,8.88,2.53,1.90,16.2,14.7,-5.0,28.95,3,80.2,13.0,eor


In [11]:
wuerzburg_data_file = "produkt_klima_monat_18810101_20231231_05705.txt"

wuerzburg_df = load_txt_from_zip(wuerzburg, wuerzburg_data_file)

wuerzburg_df

,STATIONS_ID,MESS_DATUM_BEGINN,MESS_DATUM_ENDE,QN_4,MO_N,MO_TT,MO_TX,MO_TN,MO_FK,MX_TX,MX_FX,MX_TN,MO_SD_S,QN_6,MO_RR,MX_RS,eor
0,5705,18810101,18810131,5,-999.00,-999.00,-1.80,-999.00,-999.00,-999.0,-999.0,-999.0,-999.00,-999,-999.0,-999.0,eor
1,5705,18810201,18810228,5,-999.00,-999.00,4.50,-999.00,-999.00,-999.0,-999.0,-999.0,-999.00,-999,-999.0,-999.0,eor
2,5705,18810301,18810331,5,-999.00,-999.00,9.30,-999.00,-999.00,-999.0,-999.0,-999.0,-999.00,-999,-999.0,-999.0,eor
3,5705,18810401,18810430,5,-999.00,-999.00,12.60,-999.00,-999.00,-999.0,-999.0,-999.0,-999.00,-999,-999.0,-999.0,eor
4,5705,18810501,18810531,5,-999.00,-999.00,19.70,-999.00,-999.00,-999.0,-999.0,-999.0,-999.00,-999,-999.0,-999.0,eor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,5705,20230801,20230831,3,5.34,19.66,25.50,14.66,2.35,35.0,24.2,8.6,206.85,3,67.1,12.4,eor
1696,5705,20230901,20230930,3,4.01,18.49,25.80,11.89,1.80,31.7,20.9,5.6,261.82,3,16.2,5.2,eor
1697,5705,20231001,20231031,3,6.52,12.71,18.03,8.18,2.42,27.8,18.8,-0.6,132.02,3,75.9,17.2,eor
1698,5705,20231101,20231130,3,6.80,6.10,8.89,3.19,2.97,16.8,19.9,-4.6,43.52,3,76.2,13.8,eor


In [12]:
# Combine the dataframes so we can save to a database
combined_df = pd.concat([bamberg_df, wuerzburg_df])

In [13]:
# Remove eor column and replace missing and invalid values (-999) with pd.NA.
clean_df(combined_df)

combined_df

,STATIONS_ID,MESS_DATUM_BEGINN,MESS_DATUM_ENDE,QN_4,MO_N,MO_TT,MO_TX,MO_TN,MO_FK,MX_TX,MX_FX,MX_TN,MO_SD_S,QN_6,MO_RR,MX_RS
0,282,18810101,18810131,5,<NA>,<NA>,-2.7,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,282,18810201,18810228,5,<NA>,<NA>,3.5,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,282,18810301,18810331,5,<NA>,<NA>,7.2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,282,18810401,18810430,5,<NA>,<NA>,10.4,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,282,18810501,18810531,5,<NA>,<NA>,17.8,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1695,5705,20230801,20230831,3,5.34,19.66,25.5,14.66,2.35,35.0,24.2,8.6,206.85,3,67.1,12.4
1696,5705,20230901,20230930,3,4.01,18.49,25.8,11.89,1.8,31.7,20.9,5.6,261.82,3,16.2,5.2
1697,5705,20231001,20231031,3,6.52,12.71,18.03,8.18,2.42,27.8,18.8,-0.6,132.02,3,75.9,17.2
1698,5705,20231101,20231130,3,6.8,6.1,8.89,3.19,2.97,16.8,19.9,-4.6,43.52,3,76.2,13.8


In [14]:
# Save the DataFrame to an SQLite database
save_to_sqlite(combined_df, 'franconian-weather')

Data saved to SQLite database successfully!


### Structure and Quality
The data is tabularly structured into means of means and extrema (min. and max.) as well as the maximum of extrema by month.

We chose to combine both weather stations into a single database for easier storage and loading. It is trivial to split the data again by station ID if need be.

Accuracy, completeness, consistency and timeliness is ensured by a trusted institution, the DWD.

The data is relevant for the question at hand. However, a major factor which affects the growth of vineyards is soil nutrients. That information is not present in our data.

### Output Format
The output is formatted as a .sqlite file for long-term storage and wide compatibility. We intend to analyze the data using pandas dataframes.

### Reflection and Potential Issues
We intend to compare these categories or features between the two datasets and see how much overlap there is.

We're conflicted on renaming the columns; on the one hand they are well-defined with precise meanings from DWD but on the other hand one needs to become familiar with this naming convention to work with them effectively.

We're also concerned that we'll run into errors when trying to compare periods with missing data. However, because a comparison is not possible, we'll likely drop the rows (months) of the specific columns (features) which cannot be compared. We're unable to do this during the data pipeline because that would create a poorly structured database (some columns will be missing some months that other columns are not missing).